#Import Models

In [ ]:
import cv2
import dlib
import numpy as np
import glob
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from keras.models import load_model

In [ ]:
!pip install git+https://github.com/giuliano-oliveira/gdown_folder.git
import gdown
gdown.download_folder('https://drive.google.com/drive/u/1/folders/11yJEd2cue34lSodhoKUzOHNbbn31sXWh', quiet=True)

  Cloning https://github.com/giuliano-oliveira/gdown_folder.git to /tmp/pip-req-build-m3ue6xli
  Running command git clone -q https://github.com/giuliano-oliveira/gdown_folder.git /tmp/pip-req-build-m3ue6xli
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=12126 sha256=9fd1fa2dd0bb4c8eefcdee45b0486db4512c44fefc36fbd97f418b8c3f1550d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-h2qu8y2g/wheels/da/5e/24/24fbdc00f3e6e91cefe494b31d53de8b91241ef924a15358dc
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4


'/content/MARCON_FF++social/val.json'

# Import  test video

In [ ]:
!gdown https://drive.google.com/uc?id=18tLnSThjqsxDs1sPl-vLt9VwqK3yp63i

Downloading...
From: https://drive.google.com/uc?id=18tLnSThjqsxDs1sPl-vLt9VwqK3yp63i
To: /content/000_003.mp4
100% 465k/465k [00:00<00:00, 16.3MB/s]


In [ ]:
# from github FaceForensics++
def get_boundingbox(face, width, height, scale=1.3):  # takes a dlib face to return a bounding box
   x1 = face.left()
   y1 = face.top()
   x2 = face.right()
   y2 = face.bottom()

   size_bb = int(max(x2 - x1, y2 - y1)*scale)   # size of bounding box 

   center_x = (x1 + x2)/2
   center_y = (y1 + y2)/2
   
   x1 = max(int(center_x - size_bb // 2), 0)
   y1 = max(int(center_y - size_bb // 2), 0)
   size_bb = min(width - x1, size_bb)
   size_bb = min(height - y1, size_bb)
   
   return x1, y1, size_bb

def preprocess_input(cropped_frame):
  img = Image.fromarray(cropped_frame)
  img = img.resize([299,299], Image.NEAREST)
  img = np.asarray(img)
  img = img / 255.0
  img = np.expand_dims(img, axis=0)
  return img

# analyze the whole video
def analyze_video(video_dir, model):
   cap = cv2.VideoCapture(video_dir)
      
   count = 0  # count frames read
   numb = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) )   # get number of frames
   prediction = []
   
   pbar = tqdm(total=numb-count, position=0, leave=True, desc='Analyzing video: ')   

   while (cap.isOpened() ):
     ret, frame = cap.read()
     pbar.update(1)

     if not ret:
        break
     
     height, width = frame.shape[:2]

     face_detector = dlib.get_frontal_face_detector()  # set face detector model
     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # convert to grayscale
     faces = face_detector(gray, 1)    # detect faces
       
     if len(faces):    # select first largest face
       face = faces[0]  
       x, y, size = get_boundingbox(face, width, height)   # get bounding box
       cropped_face = frame[y:y+size, x:x+size]    # crop face
       preprocessed_input = preprocess_input(cropped_face)  #preprocess for feeding network
       prediction.append(model.predict(preprocessed_input)[0])
     else:
       cap.set(1,count+1)       
     
     count+=1

     if (count >= numb):  #stop when all frames are processed
       break
       pbar.close()
       cap.release()
       cv2.destroyAllWindows()
  
   return np.asarray(prediction)

def decode_prediction(prediction):
  class_names = ['real', 'fake'] 
  for i, logits in enumerate(prediction):
    class_idx = tf.argmax(logits).numpy()
    p = logits[class_idx]
    name = class_names[class_idx]
    print("Frame {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))

In [ ]:
## ANALYZE VIDEO (eg. video manipulated with NeuralTextures)
# load model
model = load_model('MARCON_FF++social/Models/Xception/Xception_NeuralTextures.h5')

predictions = analyze_video('000_003.mp4', model)
decode_prediction(predictions)

Analyzing video: 100%|██████████| 396/396 [05:56<00:00,  1.11it/s]


Frame 0 prediction: real (91.0%)
Frame 1 prediction: real (84.9%)
Frame 2 prediction: real (86.9%)
Frame 3 prediction: real (91.1%)
Frame 4 prediction: real (91.7%)
Frame 5 prediction: real (90.4%)
Frame 6 prediction: real (90.4%)
Frame 7 prediction: real (92.8%)
Frame 8 prediction: real (94.1%)
Frame 9 prediction: real (88.5%)
Frame 10 prediction: real (91.3%)
Frame 11 prediction: real (77.6%)
Frame 12 prediction: real (69.2%)
Frame 13 prediction: fake (80.1%)
Frame 14 prediction: fake (96.8%)
Frame 15 prediction: fake (97.8%)
Frame 16 prediction: real (53.9%)
Frame 17 prediction: fake (73.8%)
Frame 18 prediction: fake (82.0%)
Frame 19 prediction: fake (72.9%)
Frame 20 prediction: fake (81.7%)
Frame 21 prediction: fake (90.1%)
Frame 22 prediction: fake (75.5%)
Frame 23 prediction: fake (99.7%)
Frame 24 prediction: fake (98.4%)
Frame 25 prediction: real (90.6%)
Frame 26 prediction: real (50.8%)
Frame 27 prediction: fake (94.6%)
Frame 28 prediction: fake (87.8%)
Frame 29 prediction: rea

#Exercise
Using the dataset, check the drop of perfomance when a model trained on non-shared videos is used on shared ones. Then check the opposite. \\
How could we improve the results?
You can find the models in 'MARCON_FF++social/Models/*/ . The models marked with 'YT' underwent transfer learning on Youtube shared videos. In contrast, the models marked with 'FB'underwent transferlearning on Facebook share videos.

In [ ]:
!gdown https://drive.google.com/uc?id=13bzOI4iRMCnYgrPzZG8jaH0vhl92kY6D
!unzip video_fake_face_detection.zip

In [ ]:
#YOUR CODE#